<a href="https://colab.research.google.com/github/vjmiyagi/DS-Unit-2-Kaggle-Challenge/blob/master/module3-cross-validation/VJ_Miyagi_LS_DSPT6_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


**You can't just copy** from the lesson notebook to this assignment.

- Because the lesson was **regression**, but the assignment is **classification.**
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

#  Review requirements for your portfolio project, then submit your dataset.


# [Powerball](https://data.ny.gov/api/views/d6yy-54nr/rows.csv?accessType=DOWNLOAD&sorting=true)

#  Continue to participate in our Kaggle challenge.

In [0]:
# IMPORTS

import category_encoders as ce 
import numpy as np
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [0]:
# Define function for data wrangling
def wrangle(X):
    X = X.copy()
    
    # Unwanted zeros
    col_zeros = ['latitude', 'longitude', 'construction_year', 'gps_height',
                 'population']
    
    for col in col_zeros:
        X[col] = X[col].replace(0, np.nan)
        X[col+'_MISSING'] = X[col].isnull()
    
    # Drop unwanted columns for high cardinality, duplicates and other
    # unwanted columns
    unwanted =  ['id','installer','num_private', 'recorded_by',
                 'scheme_management', 'scheme_name', 'payment', 'permit',
                 'extraction_type', 'extraction_type_group', 'funder', 'lga',
                 'management', 'management_group', 'payment_type', 
                 'public_meeting', 'quality_group', 'quantity_group', 'region',
                 'source_type', 'source', 'subvillage', 'ward', 
                 'waterpoint_type_group', 'wpt_name']
    X = X.drop(columns = unwanted)

    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)

    # Extract year from date_recorded and drop original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['year'] = X['year_recorded'] - X['construction_year']
    X['years_MISSING'] = X['year'].isnull()
    X = X.drop(columns = 'date_recorded')

    # Reduce cardinality of extraction_type_class
    x = 'extraction_type_class'
    X[x].replace('submersible', 'other', inplace=True)
    X[x].replace('motorpump', 'other', inplace=True)
    X[x].replace('rope pump', 'other', inplace=True)
    X[x].replace('wind-powered', 'other', inplace=True)

    # Reduce cardinality of quantity
    x = 'quantity'
    X[x].replace('insufficient', 'other', inplace=True)
    X[x].replace('dry', 'other', inplace=True)
    X[x].replace('seasonal', 'other', inplace=True)
    X[x].replace('unknown', 'other', inplace=True)
    
    # Reduce cardinality of waterpoint_type
    x = 'waterpoint_type'
    X[x].replace('communal standpipe multiple', 'other', inplace=True)
    X[x].replace('improved spring', 'other', inplace=True)
    X[x].replace('cattle trough', 'other', inplace=True)
    X[x].replace('dam', 'other', inplace=True)

    # Do Ordinal Encoding
    ord_enc = ce.OrdinalEncoder()
    X = ord_enc.fit_transform(X)

    # Return wrangled dataframe
    return X

In [0]:
# Prepare to concat train and test for data wrangling, set up y_train and drop
# target in process

train['train'] = 1
test['train'] = 0

# Ordinal Encode 'status_group', set as target and drop from train
oe = ce.OrdinalEncoder()
train['status_group'] = oe.fit_transform(train['status_group'])

In [115]:
train['status_group'].head()

0    1
1    1
2    1
3    2
4    1
Name: status_group, dtype: int64

In [116]:
y_train = train['status_group']
train.drop(columns='status_group', inplace=True)
train.shape, test.shape, y_train.shape

((59400, 41), (14358, 41), (59400,))

In [117]:
combo = pd.concat([train, test] ,ignore_index=True).copy()
print(combo.shape)
combo.head()

(73758, 41)


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,train
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,1
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,1
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1


In [118]:
combo = wrangle(combo)
combo.head()

,amount_tsh,gps_height,longitude,latitude,basin,region_code,district_code,population,construction_year,extraction_type_class,water_quality,quantity,source_class,waterpoint_type,train,latitude_MISSING,longitude_MISSING,construction_year_MISSING,gps_height_MISSING,population_MISSING,year_recorded,year,years_MISSING
0,6000.0,1390.0,34.938093,-9.856322,1,11,5,109.0,1999.0,1,1,1,1,1,1,False,False,False,False,False,2011,12.0,False
1,0.0,1399.0,34.698766,-2.147466,2,20,2,280.0,2010.0,1,1,2,2,1,1,False,False,False,False,False,2013,3.0,False
2,25.0,686.0,37.460664,-3.821329,3,21,4,250.0,2009.0,1,1,1,2,2,1,False,False,False,False,False,2013,4.0,False
3,0.0,263.0,38.486161,-11.155298,4,90,63,58.0,1986.0,2,1,2,1,2,1,False,False,False,False,False,2013,27.0,False
4,0.0,NaN,31.130847,-1.825359,2,18,1,NaN,NaN,1,1,2,2,1,1,False,False,True,True,True,2011,NaN,True


In [119]:
# Split combo back into train and test

# x = combo.loc[combo.train]=1
# y = combo.loc[combo.train]=0

x = combo['train']==1
y = combo['train']==0

train = combo[x]
test = combo[y]

train = train.drop(columns='train')
test = test.drop(columns='train')

train.shape, test.shape

((59400, 22), (14358, 22))

In [120]:
y_train

0        1
1        1
2        1
3        2
4        1
        ..
59395    1
59396    1
59397    1
59398    1
59399    1
Name: status_group, Length: 59400, dtype: int64

#  Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.


In [121]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

pipeline = make_pipeline(
    ce.OneHotEncoder(use_cat_names=True),
    SimpleImputer(),
    StandardScaler(),
    SelectKBest(f_regression),
    Ridge()
)

param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'],
    'selectkbest__k': range(1, len(train.columns)+1),
    'ridge__alpha': [0.1, 1, 10],
}

search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=100,
    cv=5,
    scoring='neg_mean_absolute_error',
    verbose=10,
    return_train_score=True,
    n_jobs=-1
)

search.fit(train, y_train)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   26.8s
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed:   31.0s
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed:   35.8s
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:   40.7s
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed:   

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('onehotencoder',
                                              OneHotEncoder(cols=None,
                                                            drop_invariant=False,
                                                            handle_missing='value',
                                                            handle_unknown='value',
                                                            return_df=True,
                                                            use_cat_names=True,
                                                            verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            fill_value=None,
  

In [122]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)


Best hyperparameters {'simpleimputer__strategy': 'mean', 'selectkbest__k': 21, 'ridge__alpha': 0.1}
Cross-validation MAE 0.5036086152308801


In [123]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,77,72,74,36,35,52,84,6,57,44,32,90,12,92,47,7,78,0,99,79,22,54,55,50,37,68,97,41,75,96,59,51,29,43,18,34,48,3,69,40,...,63,58,23,1,27,56,17,65,2,15,91,39,9,4,46,13,98,42,93,83,95,8,64,80,73,45,94,19,5,31,26,24,14,88,86,16,76,71,11,20
mean_fit_time,0.336189,0.345667,0.467015,0.356257,0.363511,0.504313,0.480134,0.484515,0.517536,0.363724,0.36369,0.479047,0.330573,0.493051,0.341392,0.493123,0.467208,0.536605,0.313309,0.456481,0.530184,0.335863,0.499321,0.365886,0.362483,0.476615,0.498668,0.500716,0.318728,0.711626,0.496959,0.496736,0.332385,0.337133,0.504293,0.35028,0.349214,0.485456,0.327831,0.35019,...,0.480216,0.352419,0.34724,0.342646,0.33645,0.334678,0.476302,0.474339,0.478723,0.324824,0.322314,0.336365,0.475897,0.333857,0.338657,0.320679,0.448676,0.490423,0.572889,0.32498,0.648911,0.315784,0.47476,0.459328,0.449994,0.336361,0.643742,0.350591,0.48296,0.329232,0.338198,0.4985,0.462532,0.470536,0.468673,0.476677,0.468663,0.320823,0.330004,0.342284
std_fit_time,0.0134879,0.0138568,0.00636788,0.0199136,0.0184838,0.0103267,0.0113696,0.00911325,0.0152217,0.033008,0.00758181,0.00981631,0.0131824,0.0065137,0.00928811,0.00471628,0.0138746,0.0297039,0.00598924,0.0104579,0.0116497,0.00773133,0.0254685,0.00839161,0.0156888,0.012028,0.156738,0.00705892,0.00476931,0.0569537,0.00735863,0.0112614,0.012044,0.0140168,0.019857,0.0152358,0.00706303,0.0166333,0.015144,0.0172347,...,0.0155011,0.0108248,0.01086,0.0102028,0.00740717,0.00757282,0.0224399,0.00583266,0.0152328,0.00911293,0.00976157,0.0110456,0.0118363,0.0191531,0.00838108,0.00834124,0.0104461,0.0174204,0.117631,0.00953734,0.014074,0.00567706,0.0159185,0.00964336,0.0126615,0.0228076,0.0108223,0.0123764,0.014794,0.00734736,0.0133351,0.0259363,0.00812026,0.0129454,0.00603649,0.0165901,0.0199315,0.0189241,0.00946697,0.00956534
mean_score_time,0.0378466,0.0373728,0.0373188,0.0396863,0.0400446,0.038601,0.0359406,0.0358849,0.0421981,0.0392868,0.0386493,0.0379808,0.0373294,0.0379464,0.0381624,0.0365394,0.0401595,0.0402301,0.036789,0.0398109,0.039052,0.0457137,0.0399901,0.0411707,0.0380311,0.036635,0.0530147,0.0387639,0.037637,0.0800961,0.0382742,0.0400986,0.0377409,0.0417988,0.0424389,0.0378087,0.039274,0.0376794,0.0400312,0.0421605,...,0.0379666,0.0408916,0.0390301,0.0381299,0.0378365,0.0384507,0.0371133,0.0381934,0.0378164,0.0363935,0.0384625,0.037426,0.0390134,0.0370634,0.0396148,0.0408916,0.038747,0.042813,0.0509264,0.0365012,0.0714742,0.0372789,0.0382783,0.03878,0.035879,0.0389137,0.0710164,0.0389913,0.0355416,0.0379522,0.0377006,0.0379077,0.0362907,0.0355332,0.0358895,0.037122,0.0363153,0.0360172,0.0364613,0.040212
std_score_time,0.00141324,0.00013689,0.000352243,0.00290897,0.000756042,0.00121395,0.000717787,0.00168367,0.00193878,0.00122964,0.000872933,0.00104188,0.00103408,0.00096141,0.000692338,0.00191163,0.00512675,0.00135919,0.000956643,0.00590215,0.00100714,0.0097796,0.00208739,0.00363138,0.00043411,0.0023184,0.0176783,0.00251393,0.00229604,0.00942991,0.00342253,0.00282014,0.00139578,0.00420019,0.0062122,0.00133932,0.0014113,0.00120358,0.00823922,0.00652408,...,0.00228668,0.00397174,0.00160264,0.00170512,0.00145105,0.00172284,0.000958909,0.000885288,0.000503699,0.000435815,0.00402316,0.00204795,0.00197737,0.00101518,0.00134671,0.00550704,0.00474592,0.00727456,0.0172851,0.00150338,0.00315578,0.00132344,0.00131958,0.00575252,0.00142323,0.00334649,0.00101852,0.00143269,0.00145354,0.00119804,0.000960352,0.00170988,0.00115969,0.000654459,0.00084057,0.00138387,0.000688793,0.00108128,0.000769659,0.00345233
param_simpleimputer__strategy,mean,mean,median,mean,mean,median,median,median,median,mean,mean,median,mean,median,mean,median,median,median,mean,median,median,mean,median,mean,mean,median,mean,median,mean,mean,median,median,mean,mean,median,mean,mean,median,mean,mean,...,median,mean,mean,mean,mean,mean,median,median,median,mean,mean,mean,median,mean,mean,mean,median,median,median,mean,mean,mean,median,median,median,mean,mean,mean,median,mean,mean,median,median,median,median,median,medi

In [0]:
pipeline = search.best_estimator_

In [0]:
from sklearn.metrics import mean_absolute_error

y_pred = pipeline.predict(test)

In [126]:
y_pred

array([1.37042621, 1.50588232, 1.44278582, ..., 1.52403319, 1.52003746,
       1.63807676])

In [0]:
y = pd.DataFrame(data=y_pred)

In [128]:
y.head()

,0
0,1.370426
1,1.505882
2,1.442786
3,1.898843
4,1.410760


In [129]:
y[0].describe()

count    14358.000000
mean         1.526796
std          0.201440
min          0.177466
25%          1.389291
50%          1.522725
75%          1.661783
max          2.170026
Name: 0, dtype: float64

In [130]:
y_train.head()

0    1
1    1
2    1
3    2
4    1
Name: status_group, dtype: int64

#  Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue Submit Predictions

In [0]:
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('your-submission-filename.csv', index=False)

# Commit your notebook to your fork of the GitHub repo.